In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
from configparser import ConfigParser
import psycopg2
import psycopg2.extras as extras


def get_postgres_conn(**kwargs):
 "database=dbname_str,user=user_str,password=pwd_str,host=host_str,port=port_number_str"   
 try:
#   conn = psycopg2.connect(
#          database="SMartDB", user='postgres', password='P@ssw0rd', host='localhost', port='5432'
#      )
                     
    conn = psycopg2.connect(
         database=kwargs['database'], user=kwargs['user'], password=kwargs['password'],host=kwargs['host'], port=kwargs['port']
     )
    
    return conn
 except Exception as error:
  raise error



def add_data_values(conn, df, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    #return query,tuples
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        raise error
        return 0
    print("importing data succeeded")
    return 1
    cursor.close()
    
def get_one_sql(conn,sql_cmd, params):

    row = None
    item=None
    try:
        cur = conn.cursor()
        if params is not None:
          cur.execute(sql_cmd, params)
        else:
          cur.execute(sql_cmd)

        #print("The number of cost: ", cur.rowcount)

        fields = [col[0] for col in cur.description]
        if cur.rowcount >0:
         item = dict(zip(fields, cur.fetchone()))

        #row = cur.fetchone()
        #         while row is not None:
        #             print(row)
        #             row = cur.fetchone()

        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        raise error
    finally:
        if conn is not None:
            conn.close()
    #return dict

    return item


def get_list_sql(conn,sql_cmd, params):
    itemList=None
    try:
        cur = conn.cursor()

        if params is not None:
            cur.execute(sql_cmd, params)
        else:
            cur.execute(sql_cmd)

        #print("The number of cost: ", cur.rowcount)
        fields = [col[0] for col in cur.description]
        if cur.rowcount > 0:
         itemList=[dict(zip(fields, rw)) for rw in cur.fetchall()]

        # for row in rows:
        #     print(row)

        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        raise error
    finally:
        if conn is not None:
            conn.close()
    # return list(dict)
    return itemList



In [ ]:
# Excel file should be the same location as python file
#file_path=r'D:\JupyterCode\Yit\IncidentReport\SMartApp_Dev\EmployessFiles\import_employee_master.xlsx'
file_path='import_employee_master.xlsx'
sheet_name='NCS '

# load scheema column from app_employee except id  and map to column in excel file
cols=['employee_nickname','employee_name','employee_position','employee_telephone','employee_email']

# specify columns to inspect special chars
cols_remove_xchars=['employee_name']



In [ ]:
startwith_chars_1=['Mr.','Mrs.','Miss.','Ms.']
startwith_chars_2=['Mr ','Mrs ','Miss ','Ms ']
def remove_xchar(item):
  for cx in   startwith_chars_1: 
    if item.startswith(cx):
       item = item.replace(cx,'')
       item=item.strip()
       return item
    
  for cx in   startwith_chars_2: 
    if item.startswith(cx):
       item = re.sub(f'\\b{cx}\\d*\\b', '', item)
       item=item.strip()
       return item

  return item 

# new_item=remove_xchar('Miss. Wannaporn Saw-Khow')
# print(new_item)
    
    

In [ ]:

try:
    df=pd.read_excel(file_path,sheet_name=sheet_name,usecols=cols)
    df=df.replace(np.nan, '', regex=True)  
   
    for c in cols:
      try:  
       df[c]=df[c].str.strip()
      except Exception as error:
       print(error)     
 
    for xc in cols_remove_xchars:
      try:  
       df[xc]=df[xc].apply(remove_xchar) 
      except Exception as error:
       print(error)   
    

    conn=get_postgres_conn(database="SMartDB", user='postgres', password='P@ssw0rd', host='localhost', port='5432')
    rslt=add_data_values(conn,df,"app_employee")
    if rslt==1:   
        print(df)

except Exception as error:
     raise error

